In [1]:
# Used the embeddings fro HuggingFace
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Load the model
from llama_index.llms.ollama import Ollama

llm = Ollama(model="phi3", request_timeout=120.0)
# Configure the settings to so that the desired llm and embedding_model is used.
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [2]:
from llama_index.readers.github import GithubRepositoryReader, GithubClient
import nest_asyncio
nest_asyncio.apply()
from llama_index.core import VectorStoreIndex
from llama_index.core import (ServiceContext, StorageContext, load_index_from_storage)

In [3]:
import os
folder_path = "./storage"
folder_exists = os.path.exists(folder_path) and os.path.isdir(folder_path)
folder_exists

True

In [4]:
if folder_exists:
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)
else:

    client = github_client = GithubClient(github_token="", verbose=False)

    reader = GithubRepositoryReader(
        github_client=github_client,
        owner="Arize-ai",
        repo="phoenix",
        use_parser=False,
        verbose=True,
        filter_directories=(
            ["docs"],
            GithubRepositoryReader.FilterType.INCLUDE,
        ),
        filter_file_extensions=(
            [
                ".png",
                ".jpg",
                ".jpeg",
                ".gif",
                ".svg",
                ".ico",
                "json",
                ".ipynb",
            ],
            GithubRepositoryReader.FilterType.EXCLUDE,
        ),
    )

    branch_documents = reader.load_data(branch="main")
    index = VectorStoreIndex.from_documents(branch_documents)
    index.storage_context.persist()

In [5]:
query_engine = index.as_query_engine()

In [6]:
question = "What is phoenix?"
prompt = f"Respond to this question:\n\n{question}\n\nAnswer:\n"
response = query_engine.query(prompt)


In [11]:
print(response)

 AI Observability Library
